# Real-world data experiment

In [38]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import Ridge
from src.thirdparties.mord.regression_based import OrdinalRidge

In [39]:
np.random.seed(0)

# Loading data

In [40]:
n_averaging_locations = 2
n_bins = 5

In [41]:
df_LRCS = joblib.load('data/batteries/dfs/df_LRCS_bins_'+str(n_bins)+'.pkl')
df_LRCSWMG = joblib.load('data/batteries/dfs/df_LRCSWMG_20221026_21457_17512.pkl')

In [42]:
n_CG_LRCS = len(df_LRCS['Comma gap (µm)'].unique())
n_ML_LRCS = n_bins
dom_CG_LRCS = [75,100,200]
dom_ML_LRCS = np.arange(n_ML_LRCS)

# Learning before abstraction

In [43]:
def eval_metric(model,Xte,yte,Xout,roundpred=True):
    if roundpred:
        preds = np.round(model.predict(Xte.reshape(-1,1)))
    else:
        preds = model.predict(Xte.reshape(-1,1))
    mses = (preds - yte)**2
    print('MSE (with {0} out): {1} ({2})'.format(Xout,np.mean(mses),np.std(mses)))
    print('${0:.2f}\pm{1:.2f}$'.format(np.mean(mses),np.std(mses)))
    
    return np.mean(mses)

## Learning on LRCS data: standard KFold

### Ordinal Regression 

In [44]:
scores = []

for tr,te in kfold.split(df_LRCS):
    Xtr,Xte = np.array(df_LRCS['Comma gap (µm)'].iloc[tr]),np.array(df_LRCS['Comma gap (µm)'].iloc[te])
    ytr,yte = np.array(df_LRCS['binned ML'].iloc[tr]),np.array(df_LRCS['binned ML'].iloc[te])
    model = OrdinalRidge().fit(Xtr.reshape(-1,1),ytr)
    score = eval_metric(model,Xte,yte,cg,False)
    scores.append(score)

print('Overall MSE: {0} ({1})'.format(np.mean(scores),np.std(scores)))

MSE (with 200 out): 0.0 (0.0)
$0.00\pm0.00$
MSE (with 200 out): 0.0 (0.0)
$0.00\pm0.00$
MSE (with 200 out): 0.07692307692307693 (0.26646935501059654)
$0.08\pm0.27$
MSE (with 200 out): 0.15384615384615385 (0.36080121229410994)
$0.15\pm0.36$
MSE (with 200 out): 0.0 (0.0)
$0.00\pm0.00$
Overall MSE: 0.046153846153846156 (0.06153846153846154)


### Ridge

In [45]:
scores = []

for tr,te in kfold.split(df_LRCS):
    Xtr,Xte = np.array(df_LRCS['Comma gap (µm)'].iloc[tr]),np.array(df_LRCS['Comma gap (µm)'].iloc[te])
    ytr,yte = np.array(df_LRCS['binned ML'].iloc[tr]),np.array(df_LRCS['binned ML'].iloc[te])
    model = Ridge().fit(Xtr.reshape(-1,1),ytr)
    score = eval_metric(model,Xte,yte,cg,True)
    scores.append(score)

print('Overall MSE: {0} ({1})'.format(np.mean(scores),np.std(scores)))

MSE (with 200 out): 0.0 (0.0)
$0.00\pm0.00$
MSE (with 200 out): 0.0 (0.0)
$0.00\pm0.00$
MSE (with 200 out): 0.07692307692307693 (0.26646935501059654)
$0.08\pm0.27$
MSE (with 200 out): 0.15384615384615385 (0.36080121229410994)
$0.15\pm0.36$
MSE (with 200 out): 0.0 (0.0)
$0.00\pm0.00$
Overall MSE: 0.046153846153846156 (0.06153846153846154)


## Learning on LRCS data: one-X out

We define the input data ($X$) and the output data ($y$).

In [46]:
def select_one_X_out(X,y,cond):
    Xte = np.array(X[cond])
    yte = np.array(y[cond])

    Xtr = np.array(X[np.logical_not(cond)])
    ytr = np.array(y[np.logical_not(cond)])
    
    return Xtr,ytr,Xte,yte

### Ordinal Regression 

In [47]:
scores = []

for cg in dom_CG_LRCS:
    cond = df_LRCS['Comma gap (µm)']==cg
    
    Xtr,ytr,Xte,yte = select_one_X_out(df_LRCS['Comma gap (µm)'],df_LRCS['binned ML'],cond)
    
    model = OrdinalRidge().fit(Xtr.reshape(-1,1),ytr)
    score = eval_metric(model,Xte,yte,cg,False)
    scores.append(score)
    
print('Overall MSE: {0} ({1})'.format(np.mean(scores),np.std(scores)))

MSE (with 75 out): 1.375 (0.9921567416492215)
$1.38\pm0.99$
MSE (with 100 out): 1.0 (0.0)
$1.00\pm0.00$
MSE (with 200 out): 3.875 (0.5994789404140899)
$3.88\pm0.60$
Overall MSE: 2.0833333333333335 (1.2761160692594629)


### Ridge

In [48]:
scores = []

for cg in dom_CG_LRCS:
    cond = df_LRCS['Comma gap (µm)']==cg
    
    Xtr,ytr,Xte,yte = select_one_X_out(df_LRCS['Comma gap (µm)'],df_LRCS['binned ML'],cond)
    
    model = Ridge().fit(Xtr.reshape(-1,1),ytr)
    score = eval_metric(model,Xte,yte,cg,True)
    scores.append(score)
    
print('Overall MSE: {0} ({1})'.format(np.mean(scores),np.std(scores)))

MSE (with 75 out): 1.375 (0.9921567416492215)
$1.38\pm0.99$
MSE (with 100 out): 1.0 (0.0)
$1.00\pm0.00$
MSE (with 200 out): 4.208333333333333 (0.9991315673568165)
$4.21\pm1.00$
Overall MSE: 2.194444444444444 (1.4322401084996403)


# Learning after abstraction

## Learning on LRCS+WMG data: standard KFold

### Ordinal Regression

In [49]:
scores = []

for tr,te in kfold.split(df_LRCSWMG):
    Xtr,Xte = np.array(df_LRCSWMG['Comma gap (µm)'].iloc[tr]),np.array(df_LRCSWMG['Comma gap (µm)'].iloc[te])
    ytr,yte = np.array(df_LRCSWMG['binned ML'].iloc[tr]),np.array(df_LRCSWMG['binned ML'].iloc[te])
    model = OrdinalRidge().fit(Xtr.reshape(-1,1),ytr)
    score = eval_metric(model,Xte,yte,cg,False)
    scores.append(score)

print('Overall MSE: {0} ({1})'.format(np.mean(scores),np.std(scores)))

MSE (with 200 out): 0.21311475409836064 (0.6036705783449827)
$0.21\pm0.60$
MSE (with 200 out): 0.5409836065573771 (1.3498484491959937)
$0.54\pm1.35$
MSE (with 200 out): 0.0 (0.0)
$0.00\pm0.00$
MSE (with 200 out): 0.2 (0.8717797887081347)
$0.20\pm0.87$
MSE (with 200 out): 0.4666666666666667 (0.8055363982396381)
$0.47\pm0.81$
Overall MSE: 0.2841530054644809 (0.19603279632935044)


### Ridge

In [50]:
scores = []

for tr,te in kfold.split(df_LRCSWMG):
    Xtr,Xte = np.array(df_LRCSWMG['Comma gap (µm)'].iloc[tr]),np.array(df_LRCSWMG['Comma gap (µm)'].iloc[te])
    ytr,yte = np.array(df_LRCSWMG['binned ML'].iloc[tr]),np.array(df_LRCSWMG['binned ML'].iloc[te])
    model = Ridge().fit(Xtr.reshape(-1,1),ytr)
    score = eval_metric(model,Xte,yte,cg,False)
    scores.append(score)

print('Overall MSE: {0} ({1})'.format(np.mean(scores),np.std(scores)))

MSE (with 200 out): 0.13164812394327538 (0.370136572219468)
$0.13\pm0.37$
MSE (with 200 out): 0.5915565512889581 (1.425455088899457)
$0.59\pm1.43$
MSE (with 200 out): 0.0001680010795951243 (2.710505431213761e-20)
$0.00\pm0.00$
MSE (with 200 out): 0.22041611899624744 (0.8720870919903739)
$0.22\pm0.87$
MSE (with 200 out): 0.23215033858235312 (0.6777670366670484)
$0.23\pm0.68$
Overall MSE: 0.23518782677808586 (0.19656087404634068)


## Learning on LRCS+WMG data: one-X out with WMG providing the missing support

We define the input data ($X$) and the output data ($y$) considering first the possibility that WMG offers support for the out-of-sample data ($CG=100$).

### Ordinal Regression 

In [51]:
scores = []

for cg in dom_CG_LRCS:
    cond0 = df_LRCS['Comma gap (µm)']==cg
    cond1 = list(cond0) + [False]*(len(df_LRCSWMG)-len(cond0))
    
    Xtr,ytr,Xte,yte = select_one_X_out(df_LRCSWMG['Comma gap (µm)'],df_LRCSWMG['binned ML'],cond1)
    
    model = OrdinalRidge().fit(Xtr.reshape(-1,1),ytr)
    score = eval_metric(model,Xte,yte,cg,False)
    scores.append(score)
    
print('Overall MSE: {0} ({1})'.format(np.mean(scores),np.std(scores)))

MSE (with 75 out): 1.375 (0.9921567416492215)
$1.38\pm0.99$
MSE (with 100 out): 0.0 (0.0)
$0.00\pm0.00$
MSE (with 200 out): 0.041666666666666664 (0.19982631347136331)
$0.04\pm0.20$
Overall MSE: 0.47222222222222227 (0.6385868851429249)


### Ridge

In [52]:
scores = []

for cg in dom_CG_LRCS:
    cond0 = df_LRCS['Comma gap (µm)']==cg
    cond1 = list(cond0) + [False]*(len(df_LRCSWMG)-len(cond0))
    
    Xtr,ytr,Xte,yte = select_one_X_out(df_LRCSWMG['Comma gap (µm)'],df_LRCSWMG['binned ML'],cond1)
    
    model = Ridge().fit(Xtr.reshape(-1,1),ytr)
    score = eval_metric(model,Xte,yte,cg,False)
    scores.append(score)
    
print('Overall MSE: {0} ({1})'.format(np.mean(scores),np.std(scores)))

MSE (with 75 out): 0.5466692789717083 (0.6854363128035611)
$0.55\pm0.69$
MSE (with 100 out): 0.00010349416644844821 (0.0)
$0.00\pm0.00$
MSE (with 200 out): 0.11555391895234014 (0.29307740386167275)
$0.12\pm0.29$
Overall MSE: 0.22077556403016563 (0.2352122944981268)


## Learning on LRCS+WMG data: one X-out with WMG not providing the missing support

### Mord Ordinal Regression 

In [53]:
scores = []

for cg in dom_CG_LRCS:
    cond = df_LRCSWMG['Comma gap (µm)']==cg
    
    Xtr,ytr,Xte,yte = select_one_X_out(df_LRCSWMG['Comma gap (µm)'],df_LRCSWMG['binned ML'],cond)
    
    model = OrdinalRidge().fit(Xtr.reshape(-1,1),ytr)
    score = eval_metric(model,Xte,yte,cg,False)
    scores.append(score)
    
print('Overall MSE: {0} ({1})'.format(np.mean(scores),np.std(scores)))

MSE (with 75 out): 1.0789473684210527 (0.480218094497228)
$1.08\pm0.48$
MSE (with 100 out): 0.26903553299492383 (0.994250264053218)
$0.27\pm0.99$
MSE (with 200 out): 0.029411764705882353 (0.16895772489817729)
$0.03\pm0.17$
Overall MSE: 0.45913155537395295 (0.44906096034554)


### Ridge

In [54]:
scores = []

for cg in dom_CG_LRCS:
    cond = df_LRCSWMG['Comma gap (µm)']==cg
    
    Xtr,ytr,Xte,yte = select_one_X_out(df_LRCSWMG['Comma gap (µm)'],df_LRCSWMG['binned ML'],cond)
    
    model = Ridge().fit(Xtr.reshape(-1,1),ytr)
    score = eval_metric(model,Xte,yte,cg,False)
    scores.append(score)
    
print('Overall MSE: {0} ({1})'.format(np.mean(scores),np.std(scores)))

MSE (with 75 out): 0.522634683585434 (0.3773472391605363)
$0.52\pm0.38$
MSE (with 100 out): 0.5683329594068245 (1.420643441232057)
$0.57\pm1.42$
MSE (with 200 out): 7.9793165139722415 (1.1118429267436363)
$7.98\pm1.11$
Overall MSE: 3.0234280523215005 (3.504391998273545)
